# Кратчайшее растояние на графе

Задача поиска самого короткого расстояния на графе встречается очень часто и сама по себе очень естественна - дорожная сеть может быть представлена в виде графа, узлы в интернет сети также представляются некоторым графом и для их анализа часто требуется найти наименьшее растояние от одного узла до другого.

In [1]:
#include <fstream>
#include <vector>
#include <iostream>
#include <algorithm>
#include <queue> 
#include <climits>
#include <set>

In [2]:
void print_file(std::string f) { // печать файла для наглядности
    std::ifstream file(f);
    std::string s;
    while(std::getline(file, s)) {
        std::cout << s << std::endl;
    }
}

In [3]:
class Vertex;

class Edge { // класс ребра
public:
    Edge(Vertex* vert) : vertex(vert), lenght(1) {}
    Edge(Vertex* vert, int len) : vertex(vert), lenght(len) {}
    ~Edge() {}
    
    Vertex* vertex;
    int lenght;
};

class Vertex { // класс вершины
public:
    Vertex(int i) : id(i), value(0) {}
    ~Vertex(){
        for(Edge* e : neighbours) delete e;
    }

    void connectOneside(Vertex * vertex, int len) {
        neighbours.push_back(new Edge(vertex, len));
    }

    void connectTwoside(Vertex * vertex, int len) {
        neighbours.push_back(new Edge(vertex, len));
        vertex->connectOneside(this, len);
    }

    int id;
    int value; // ячейка для хранения какого-то значения в вершине
    std::vector<Edge*> neighbours; // вершины, с которыми связана эта вершина
};

class Graph { // класс графа
public:
    Graph() {}
    Graph(int size) { // создать граф размена size
        vertexes.reserve(size);
        for(int i = 0; i < size; i++) {
            vertexes.push_back(new Vertex(i));
        }
    }
    
    Graph(std::string path) {
        readFromFile(path);
    }
    ~Graph(){
        for(Vertex * v : vertexes) delete v;
    }

    Vertex * at(int index) { // получить вершину по id
        return vertexes.at(index);
    }

    void readFromFile(std::string path) { // Формат: 2 -> 3 или 2 <-> 3. Это означает, что 2 вершина соединена с 3
        std::ifstream file(path);
        
        int size;
        file >> size;
        
        vertexes.reserve(size);
        for(int i = 0; i < size; i++) {
            vertexes.push_back(new Vertex(i+1));
        }

        int id1, id2, len;
        std::string type;
        while(file >> id1 >> type >> id2 >> len) {
            if(type == "->") vertexes[id1-1]->connectOneside(vertexes[id2-1], len);
            else if(type == "<->") vertexes[id1-1]->connectTwoside(vertexes[id2-1], len);
        }
    }

    std::vector<Vertex*> vertexes; // вершины графа
};

# Волновой алгоритм

Самым простым и понятным алгоритмом является волновой алгоритм. Для его работы растояния между соседними вершинами должно равнятся 1 - то есть под расстоянием между двумя вершинами мы понимаем количество переходов по вершинам графа. 

Идея данного алгоритма следующая - 
* Из начальной точки мы запускаем "волну". То есть как в поиске в ширину мы сделаем шаг во все соседние вершины. 
* Пометим, что расстояние до них равняется 1. 
* Далее из этих вершин сделаем еще один шаг в те вершины, до которых мы еще не доходили - расстояние до них будет равнятся 2. 
* И будем распространять эту "волну" до тех пор, пока не обойдем весь граф.

Иллюстрация работы алгоритма:
<img src="https://pp.userapi.com/c626120/v626120858/52390/zwzvC9rp94E.jpg">

In [4]:
void wave_search(Graph& g, Vertex* start) {
    for(Vertex * s : g.vertexes) s->value = -1; // вершины, которые мы еще не посетили
    std::queue<Vertex *> q; // очередь вершин, которые нужно пройти
    q.push(start);
    q.front()->value = 0; // начальная вершина
    while(!q.empty()) {
        Vertex * v = q.front(); // берем следующую вершину
        q.pop();
        
        for(Edge * w : v->neighbours) {
            if(w->vertex->value == -1) {
                w->vertex->value = v->value + 1; // помечаем вершину
                q.push(w->vertex); // добавляем в очередь к просмотру
            }
            // иначе ничего не делаем с вершиной
        }
    }
}

<img src="https://upload.wikimedia.org/wikipedia/commons/5/5b/6n-graf.svg">

Автор: <a href="//commons.wikimedia.org/wiki/User:AzaToth" title="User:AzaToth">User:AzaToth</a> - <a href="//commons.wikimedia.org/wiki/File:6n-graf.png" title="File:6n-graf.png">Image:6n-graf.png</a> simlar input data, Общественное достояние, <a href="https://commons.wikimedia.org/w/index.php?curid=820489">Ссылка</a>

Граф 1

In [5]:
print_file("files/6.graph"); // граф 1 (рисунок выше)

Graph g6("files/6.graph");

6
1 <-> 2 1
1 <-> 5 1
2 <-> 5 1
2 <-> 3 1
4 <-> 5 1
4 <-> 3 1
4 <-> 6 1


(Graph &) @0x7f2c1da29010


In [6]:
wave_search(g6, g6.vertexes[0]); // найдем пути из первой вершины

(void) nullptr


In [7]:
for(Vertex * v : g6.vertexes) {
    std::cout << "From vertex 1 to " << v->id << " " << v->value << " steps" << std::endl;
}

From vertex 1 to 1 0 steps
From vertex 1 to 2 1 steps
From vertex 1 to 3 2 steps
From vertex 1 to 4 2 steps
From vertex 1 to 5 1 steps
From vertex 1 to 6 3 steps


Данный алгоритм является одним из самый простых в реализации и одним из самых быстрых, если требуется найти кратчайшее расстояние от одной вершины до всех остальных. Однако он накладывает определеные ограничения на граф. Для решения этой же задачи на более сложных графах используют более продвинутые алгоритмы.

# Алгоритм Дейкстры

Данный алгоритм способен найти кратчайший путь из одной вершины до всех остальных, однако теперь растояние между двумя соседними вершиными может быть произвольным (но больше 0). 

Идея данного алгоритма очень похожа на волновой алгоритм - мы также идем из вершины в соседние и выставляем им расстояние до них. Однако теперь меняется подход того, как именно мы выбираем следующую вершину:

* Изначально выставим бесконечно большое расстояние до всех вершин, кроме начальной - ей выставим 0
* Далее возьмем вершину с минимальным расстоянием до нее (на первой итерации это и будет начальная)
* Пометим ее, как пройденную и далее будем пересчитывать расстояние для всех ее соседей, в которые мы еще не приходили
* Если вычисленное до этого расстояние до соседа меньше, чем расстояние до текущей + длина самого ребра, то это значит, что мы нашли более короткий путь до этого соседа => обновим расстояние до него.
* Будем продолжать это делать до тех пор, пока еще есть не пройденные вершины

In [8]:
class vertex_comparator {
public:
    bool operator() (const Vertex* a, const Vertex* b) const {
        if(a->value == b->value) return a < b;
        return a->value < b->value;
    }
};

In [9]:
void dijkstra_search(Graph& g, Vertex* start) {
    for(Vertex * s : g.vertexes) s->value = INT_MAX;
    start->value = 0;
    std::set<Vertex*, vertex_comparator> priority_set(g.vertexes.begin(), g.vertexes.end()); // множество позволит
    // быстро получать минимальный элемент 
    // вместо него также можно было использовать очередь с приоритетом
    while(priority_set.size() > 0) { 
        Vertex* v = *priority_set.begin();
        priority_set.erase(priority_set.begin());
        
        for(Edge * w : v->neighbours) { 
            if(priority_set.count(w->vertex) > 0) { // если эта вершина еще не просмотрена
                if(w->vertex->value > v->value + w->lenght) { // если шали путь быстрее
                    priority_set.erase(w->vertex); // обновляем значение 
                    w->vertex->value = v->value + w->lenght;
                    priority_set.insert(w->vertex);
                }
            }
            // иначе ничего не делаем с вершиной
        }
    }
}

<img src="https://upload.wikimedia.org/wikipedia/commons/d/de/Dijkstra_graph0.PNG">
Автор: Anatoly Gorbunov - <span class="int-own-work" lang="ru">собственная работа</span>, <a href="http://creativecommons.org/licenses/by-sa/3.0/" title="Creative Commons Attribution-Share Alike 3.0">CC BY-SA 3.0</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=751862">Ссылка</a>

Граф 2

In [10]:
print_file("files/7.graph"); // граф 2 (рисунок выше)

Graph g7("files/7.graph");

6
1 <-> 2 7
1 <-> 3 9
1 <-> 6 14
2 <-> 3 10
2 <-> 4 15
3 <-> 4 11
3 <-> 6 2
4 <-> 5 6
5 <-> 6 9


(Graph &) @0x7f2c1da29040


In [11]:
dijkstra_search(g7, g7.vertexes[0]); // найдем пути из первой вершины

(void) nullptr


Демонстрация работы данного алгоритма на графе 2
<img src="https://upload.wikimedia.org/wikipedia/commons/5/57/Dijkstra_Animation.gif">
Автор: <a href="//commons.wikimedia.org/w/index.php?title=User:Ibmua&amp;action=edit&amp;redlink=1" class="new" title="User:Ibmua (page does not exist)">Ibmua</a> - Work by uploader., Общественное достояние, <a href="https://commons.wikimedia.org/w/index.php?curid=6282617">Ссылка</a>

In [12]:
for(Vertex * v : g7.vertexes) {
    std::cout << "From vertex 1 to " << v->id << " " << v->value << " steps" << std::endl;
}

From vertex 1 to 1 0 steps
From vertex 1 to 2 7 steps
From vertex 1 to 3 9 steps
From vertex 1 to 4 20 steps
From vertex 1 to 5 20 steps
From vertex 1 to 6 11 steps


# Алгоритм Флойда-Уоршелла

Предыдущие алгориты решали задачу нахождения наикротчайшего пути из одной точки во все остальные. Однако иногда требуется знать кратчайший путь из каждой вершины в каждую. Данную задачу можно решить используя тот же алгоритм дейкстры для каждой вершины, однако данный алгоритм позволяет решить данную задачу эффективнее.

Идея алгоритма в следующем - он строит специальную матрицу, в которой на i, j месте будет стоять минимальное расстояние от i-той вершины до j-той.

* Изначально берется изначальная смежная матрица
* Далее она модифицируется следующим образом 
 * Производится n итераций.
 * На каждой итерации пересчитывается каждая клетка этой матрицы следующим образом: <img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/676d21e5f1ba61d05ac7ad931c84d15cd3e95735">
* По завершении итераций мы получим матрицу с минимальными расстояниями

In [13]:
class DenceGraph {
public:
    DenceGraph(std::string path) {
        readFromFile(path);
    }
    ~DenceGraph(){}
    
    void readFromFile(std::string path) { // Формат: 2 -> 3 или 2 <-> 3. Это означает, что 2 вершина соединена с 3
        std::ifstream file(path);
        
        int size;
        file >> size;
        
        vertexes = std::vector< std::vector<int> > (size, std::vector<int>(size, INT_MAX/3));
        for(int i = 0; i < size; i++) vertexes[i][i] = 0;

        int id1, id2, len;
        std::string type;
        while(file >> id1 >> type >> id2 >> len) {
            vertexes[id1-1][id2-1] = len;
            vertexes[id2-1][id1-1] = len;
        }
    }
    
    std::vector<std::vector<int>> vertexes;
};

In [14]:
void floyd_warshall_search(DenceGraph &g) {
    int n = g.vertexes.size();
    for(int k = 0; k < n; k++) {
        for(int i = 0; i < n; i++) {
            for(int j = 0; j < n; j++) {
                g.vertexes[i][j] = std::min(g.vertexes[i][j], g.vertexes[i][k]+g.vertexes[k][j]);
            }
        }
    }
}

In [15]:
print_file("files/7.graph"); // будет использовать тот же граф 2 (рисунок выше)

DenceGraph g8("files/7.graph");

6
1 <-> 2 7
1 <-> 3 9
1 <-> 6 14
2 <-> 3 10
2 <-> 4 15
3 <-> 4 11
3 <-> 6 2
4 <-> 5 6
5 <-> 6 9


(DenceGraph &) @0x7f2c1da29060


In [16]:
floyd_warshall_search(g8);

(void) @0x7ffdfebda380


In [17]:
for(int i = 0; i < g8.vertexes.size(); i++) {
    for(int j = 0; j < g8.vertexes.size(); j++) {
        std::cout << "From vertex " << i + 1 << " to " << j + 1 << " " << g8.vertexes[i][j] << " steps" << std::endl;
    }
}

From vertex 1 to 1 0 steps
From vertex 1 to 2 7 steps
From vertex 1 to 3 9 steps
From vertex 1 to 4 20 steps
From vertex 1 to 5 20 steps
From vertex 1 to 6 11 steps
From vertex 2 to 1 7 steps
From vertex 2 to 2 0 steps
From vertex 2 to 3 10 steps
From vertex 2 to 4 15 steps
From vertex 2 to 5 21 steps
From vertex 2 to 6 12 steps
From vertex 3 to 1 9 steps
From vertex 3 to 2 10 steps
From vertex 3 to 3 0 steps
From vertex 3 to 4 11 steps
From vertex 3 to 5 11 steps
From vertex 3 to 6 2 steps
From vertex 4 to 1 20 steps
From vertex 4 to 2 15 steps
From vertex 4 to 3 11 steps
From vertex 4 to 4 0 steps
From vertex 4 to 5 6 steps
From vertex 4 to 6 13 steps
From vertex 5 to 1 20 steps
From vertex 5 to 2 21 steps
From vertex 5 to 3 11 steps
From vertex 5 to 4 6 steps
From vertex 5 to 5 0 steps
From vertex 5 to 6 9 steps
From vertex 6 to 1 11 steps
From vertex 6 to 2 12 steps
From vertex 6 to 3 2 steps
From vertex 6 to 4 13 steps
From vertex 6 to 5 9 steps
From vertex 6 to 6 0 steps


Как можно заметить, пути из первой вершины полностью совпали с результатом работы алгоритма дейкстры. Однако теперь мы получили гораздо больше информации о графе.